In [ ]:
import sys
#Create a function that writes status to log and terminal
class Tee(object):
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
    def flush(self):
        pass

#Start the python log
f = open('test.logfile', 'w')
backup = sys.stdout
sys.stdout = Tee(sys.stdout, f)

In [ ]:
#Get current working directory
cwdpath = os.getcwd()

#Check the Python version and paths you are running from
print("\n \n Running Python version " + str(sys.version_info.major) + "." + str(sys.version_info.minor) +
" in the following paths:")
for i in sys.path:
    print(i)
if sys.version_info.major != 3:
    print("\n \n This script must be run in Python 3. This is an executable script that runs in the correct
version of Python if run with the ./<script>.py command on command line, or with the full path in cron")
    print(" Exiting Python \n \n")
    exit()

In [ ]:
import os
import subprocess
import smtplib
import logging
import pandas as pd
import saspy

In [ ]:
#Latin-1
sas_cmd = "/usr/local/apps/bin/sas"

#UTF-8
sas_u8_cmd = "/usr/local/apps/bin/sas_u8"

In [ ]:
#Check if file exists
if os.path.exists(cwdpath + "/formats.sas7bcat"):
    print("\n Reading format catalog from " + cwdpath + "/formats.sas7bcat")
else:
    print("\n Creating Standard SAS Format catalog in local directory " + cwdpath )
    cmd = sas_cmd + " /trials/sas/SASformats/formats.sas"

    #Run the command with the subprocess module
    run_cmd = subprocess.Popen(cmd, shell=True,stdout=subprocess.PIPE)
    #Get the results (return code) of the process
    run_cmd.communicate()
    #Check return code - if did not return with 0 return code, exit
    if run_cmd.returncode != 0:
        print("There was a problem running " + cmd + " Return code is " + str(run_cmd.returncode) )
        exit()
    else:
        print(cmd + " ran sucessfully" )

In [ ]:
#Start the SAS session
cfgfile= cwdpath + "/sascfg_sas.py"
sas = saspy.SASsession(cfgfile=cfgfile)
sas.submit("libname loc" + cwdpath +";")
saslog=sas.submit('proc format lib=loc cntlout=loc.fmt; run;')
print(saslog)
#End the SAS session
sas._endsas()

In [ ]:
#Read in formats exported in the previous step
fmtfile=cwdpath + “/fmt.sas7bdat”
fmt = pd.read_sas(fmtfile, format=”sas7bdat”, encoding=”latin-1”)

#Read in list of studies from study-site metadata data set
df = pd.read_sas(r'/trials/hivnet/data/protocolsiteinfo.sas7bdat', format='sas7bdat', encoding="latin-1")

In [ ]:
#Select records where the variable protstat is ‘open’
open_studies = df.loc[df.protstat.str.lower()=='open']

#Show the top 10 results
print(open_studies.head())

#Get list (array) of unique studies from column 0 (first column) in the data frame
prot_array = open_studies[open_studies.columns[0]].unique()

#Sort the list (array)
prot_array.sort()

In [ ]:
html_string = "Please review the following report. Update verification code to rectify discrepancies and
resolve errors and warnings <br> ";

In [ ]:
for prot in prot_array:
    #Get all information records for each study out of the format catalog
    prot_info = fmt.loc[fmt.START.str.lower()==prot
    #Get the study directory: the value of the first record (0) for the variable LABEL
    sdir = prot_info.loc[prot_info.FMTNAME=='SASDIR'].at[0, 'LABEL']
    #Get the enrollment data set name – note that not all studies have an enrollment data set
    enr_df = prot_info.loc[prot_info.FMTNAME=='TLA_TO_ENR']
    if (enr_df.size > 0):
        enrds = enr_df.at[0, 'LABEL']
    else:
        enrds = " "

In [ ]:
if enrds != "":
    html_string = html_string + "<br> <br> <b>" + prot + " Results: </b> "
    if os.path.exists("/trials/hivnet/data/"+enrds+".sas7bdat"):
        vdir = sdir+"/code/verification/"+enrds+"/code".strip()
        vcode = vdir+"/v_"+enrds+"_autocode.sas".strip()
        vlog = vdir+"/v_"+enrds+"_autocode.log".strip()
        if os.path.exists(vcode):
            #Output directories - odir is for Linux searching, todir is for clickable link in email
            odir = sdir+"/code/verification/"+enrds+"/output".strip()
            todir = odir.replace("/trials", "file:///T:")
            #Move to the verification directory for the study
            os.chdir(vdir)
            #Run verification code in SAS UTF-8
            cmd = sas_u8_cmd + " " + vcode;
            run_cmd = subprocess.Popen(cmd, shell=True,stdout=subprocess.PIPE)
            run_cmd.communicate()
            #Check return code - if did not return with 0 return code, try again with SAS Latin-1
            if run_cmd.returncode != 0:
                cmd = sas_cmd + " " + vcode;
                run_cmd = subprocess.Popen(cmd, shell=True,stdout=subprocess.PIPE)
                run_cmd.communicate()
                if run_cmd.returncode == 0:
                    print(cmd + " ran ok. Return code is " + str(run_cmd.returncode) )
                else:
                    html_string = html_string + "<br> &nbsp &nbsp There is a problem with " + cmd +
                    html_string = html_string + “ Return code is " + str(run_cmd.returncode)
                    if (run_cmd.returncode == 1):
                        #Grep the log for warnings
                        cmd = "grep 'WARNING' " + vlog
                        #Run the grep and put the results in an object called grep_wrn
                        grep_wrn = subprocess.Popen(cmd, shell=True,stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)
                        #Read the output from grep_wrn and puts it in memory
                        output, _ = grep_wrn.communicate()
                        if grep_wrn.returncode == 0:
                            #Decode the output to a string and write to email
                            wrn_str = output.decode("latin-1")

                            html_string = html_string + "<br> " + wrn_str.replace("WARNING", "<br> &nbsp")
                        if (run_cmd.returncode > 1):
                            #Grep the log for errors
                            cmd = "grep 'ERROR' " + vlog
                            #Run the grep and put the results in an object called grep_err
                            grep_err = subprocess.Popen(cmd, shell=True,stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)
                            #Read the output from grep_err and put it in memory
                            output, _ = grep_err.communicate()
                            if grep_err.returncode == 0:
                                #Decode the output to a string and write string to email
                                err_str = output.decode("latin-1")
                                html_string = html_string + "<br> " + err_str.replace("ERROR", "<br> &nbsp")
        else:
            print(cmd + " ran ok" ) 

In [ ]:
#Import packages
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#Define parameters
to_address = "julie@scharp.org"
from_address = "julie@scharp.org"
SERVER = "localhost"

# Create message container - the correct MIME type is multipart/alternative.
msg = MIMEMultipart('alternative')
msg['Subject'] = "Enrollment Verification Summary"
msg['From'] = from_address
msg['To'] = to_address

# The body of the message (a plain-text and an HTML version).
text = html_string
html = "<html><head></head><body>"+html_string+"</body></html>"

# Record the MIME types of both parts - text/plain and text/html.
part1 = MIMEText(text, 'plain')
part2 = MIMEText(html, 'html')

# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.
msg.attach(part1)
msg.attach(part2)

# Send the message via local SMTP server.
s = smtplib.SMTP(SERVER)
# sendmail function takes 3 arguments: sender's address, recipient's address
# and message to send - here it is sent as one string.
s.sendmail(from_address, to_address, msg.as_string())
s.quit()